### 練習問題2.58
微分プログラムを修正して、+と\*が前置演算⼦で はなく中置演算⼦となるような、  
通常の数学の記法に対して動作させたいとする。  
微分プログラムは抽象データによって定義されているので、  
微分プログラムの動作基盤となる代数式の表現を定義する述語、  
セレクタ、コンストラクタを変更するだけで、異なる表現を扱うように修正できる。

- a. (x + (3 * (x + (y + 2))))のような中置記法で表された代数式を微分するにはどのようにするかを⽰せ。
     問題を簡単に するため、+と\*は常に⼆つの引数を取り、式は完全に括弧 でくくられていると仮定せよ。   
     
- b. (x + 3 * (x + y + 2))のような標準的な代数記法を認めると、  
     問題はずっと難しくなる。この記法では、必要のない括弧は省略し、  
     乗算は加算より先に⾏われると仮定している。   
     ここでの微分プログラムがそのような記法に対してもうまく動くように、  
     適切な述語、セレクタ、コンストラクタを設計 できるだろうか。


bについては回答を割愛する。  
演算子の優先順位、非演算子の取り出しなどが難しいと思われる。

In [1]:
; aの回答

(define (=number? exp num)(and (number? exp) (= exp num)))

; eは変数か?
(define (variable? x)(symbol? x)) 

; v1とv2は同じ変数か?
(define (same-variable? v1 v2)(and (variable? v1) (variable? v2) (eq? v1 v2)))

; eは和か?
(define (sum? x)(and (pair? x) (eq? (cadr x) '+)))

; 和eの加数
(define (addend s)(car s))

; 和eの被加数
(define (augend s)(caddr s))

; a1とa2の和を構築する
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list a1 '+ a2)))
  )

;eは積か?
(define (product? x)(and (pair? x) (eq? (cadr x) '*))) 

; 積eの乗数
(define (multiplier p)(car p))

;積eの被乗数
(define (multiplicand p)(caddr p))

; m1とm2の積を構築する
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list m1 '* m2))
    )
  )

;eはべき乗か?
(define (exponentiation? x)
  (and (pair? x) (eq? (cadr x) '**))) 

; べき乗の基数
(define (base x)(car x))

; 指数
(define (exponent x)(caddr x))

; べき数を構築する
(define (make-exponentiation m1 m2)
  (cond ((=number? m2 0) 1)
        ((=number? m2 1) m1)
        ;((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2) (> m2 0)) ((make-exponentiation (* m1 m1) (- m2 1))))
        (else (list m1 '** m2)))
  )


(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum (make-product (multiplier exp)
                                                (deriv (multiplicand exp) var))
                                  (make-product (deriv (multiplier exp) var) (multiplicand exp))))
        ((exponentiation? exp)
         (let ((u (base exp))
               (e (exponent exp)))
            (make-product (make-product e (make-exponentiation u (- e 1))) (deriv u var))
           )
         )
        (else (error "unknown expression type: DERIV" exp)))
  )

In [2]:
(deriv '(x * x) 'x)

(x + x)

In [3]:
(deriv '(x * (x * x)) 'x)

((x * (x + x)) + (x * x))

In [4]:
(deriv '(x + (3 * (x + (y + 2)))) 'x)

4

In [5]:
(deriv '(x ** 2) 'x)

(2 * x)

In [6]:
(deriv '(x ** 3) 'x)

(3 * (x ** 2))